In [4]:
!pip install transformers


In [5]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import os
from docx import Document
import torch
import csv
import requests
from tqdm import tqdm


In [14]:
from transformers import AutoConfig

# Load the model's configuration
config = AutoConfig.from_pretrained("dicta-il/dictabert-ner")

# Print the label mapping
print(config.id2label)


{0: 'B-ANG', 1: 'B-DUC', 2: 'B-EVE', 3: 'B-FAC', 4: 'B-GPE', 5: 'B-LOC', 6: 'B-ORG', 7: 'B-PER', 8: 'B-WOA', 9: 'B-INFORMAL', 10: 'B-MISC', 11: 'B-TIMEX', 12: 'B-TTL', 13: 'I-DUC', 14: 'I-EVE', 15: 'I-FAC', 16: 'I-GPE', 17: 'I-LOC', 18: 'I-ORG', 19: 'I-PER', 20: 'I-WOA', 21: 'I-ANG', 22: 'I-INFORMAL', 23: 'I-MISC', 24: 'I-TIMEX', 25: 'I-TTL', 26: 'O'}


In [ ]:
from tokenizers.decoders import WordPiece

# Load tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained("avichr/Legal-heBERT")
model = AutoModelForTokenClassification.from_pretrained("avichr/Legal-heBERT")

# Create an NER pipeline
# ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")
oracle = pipeline('ner', model='dicta-il/dictabert-ner', device=0, aggregation_strategy='simple')
oracle.tokenizer.backend_tokenizer.decoder = WordPiece()


Some weights of BertForTokenClassification were not initialized from the model checkpoint at avichr/Legal-heBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


In [17]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from tqdm import tqdm
from docx import Document
import os

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("dicta-il/dictabert-ner")
model = AutoModelForTokenClassification.from_pretrained("dicta-il/dictabert-ner")

# Create an NER pipeline
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

# Function to split text into chunks based on tokens
def chunk_text_by_tokens(text, max_tokens):
    paragraphs = text.split("\n")
    chunks = []
    for paragraph in paragraphs:
        tokens = tokenizer.encode(paragraph, add_special_tokens=False)
        if len(tokens) > max_tokens:
            for i in range(0, len(tokens), max_tokens):
                chunk = tokens[i:i + max_tokens]
                chunks.append(tokenizer.decode(chunk, skip_special_tokens=True))
        else:
            chunks.append(paragraph)
    return chunks

# Function to read .docx files
def read_docx(file_path):
    doc = Document(file_path)
    return "\n".join([p.text for p in doc.paragraphs])

# Process files and query the NER model
def process_files(directory, max_tokens=512):
    files = [f for f in os.listdir(directory) if f.endswith(".docx")]
    for filename in tqdm(files, desc="Processing files"):
        file_path = os.path.join(directory, filename)
        try:
            text = read_docx(file_path)
            chunks = chunk_text_by_tokens(text, max_tokens)

            results = []
            for chunk in chunks:
                try:
                    chunk_results = ner_pipeline(chunk)
                    results.extend(chunk_results)
                except Exception as e:
                    print(f"Error processing chunk: {str(e)}")

            # Filter results for TTL and DUC labels
            relevant_labels = {'TTL', 'DUC'}
            threshold = 0.8
            filtered_results = [
                entity for entity in results
                if entity['entity_group'] in relevant_labels and entity['score'] >= threshold
            ]

            # Merge entities
            merged_entities = []
            current_entity = ""
            current_label = None

            for entity in filtered_results:
                if not entity['word'].startswith("##"):
                    if current_entity and current_label:
                        merged_entities.append((current_entity.strip(), current_label))
                    current_entity = entity['word']
                    current_label = entity['entity_group']
                else:
                    current_entity += entity['word'].replace("##", "")

            if current_entity and current_label:
                merged_entities.append((current_entity.strip(), current_label))

            # Display results
            print(f"\nResults for {filename}:")
            for entity, label in merged_entities:
                print(f"Entity: {entity}, Label: {label}")

        except Exception as e:
            print(f"Error processing file {filename}: {str(e)}")

# Main script
if __name__ == "__main__":
    docs_directory = "/home/liorkob/thesis/nlp_course/lcp/docx"
    process_files(docs_directory, max_tokens=510)


Device set to use cuda:0
Processing files:   2%|▏         | 1/55 [00:01<01:16,  1.42s/it]


Results for ME-15-08-54962-361.docx:
Entity: השופט, Label: TTL
Entity: אב"ד, Label: TTL
Entity: עו"ד, Label: TTL
Entity: עו"ד, Label: TTL
Entity: עו"ד, Label: TTL
Entity: עו"ד, Label: TTL
Entity: עו"ד, Label: TTL
Entity: כב', Label: TTL
Entity: השופטת, Label: TTL
Entity: 05, Label: TTL
Entity: השופט, Label: TTL
Entity: השופט, Label: TTL
Entity: השופט, Label: TTL
Entity: השופטת, Label: TTL
Entity: השופט, Label: TTL
Entity: השופטת, Label: TTL


Processing files:   4%|▎         | 2/55 [00:02<00:57,  1.09s/it]


Results for ME-15-09-40313-52.docx:
Entity: עו"ד, Label: TTL
Entity: עוה"ד, Label: TTL
Entity: סמנכ"ל, Label: TTL
Entity: עו"ד, Label: TTL
Entity: עוה"ד, Label: TTL
Entity: קרל גוסטב, Label: DUC
Entity: כב', Label: TTL
Entity: השופטת, Label: TTL
Entity: כב', Label: TTL
Entity: השופט, Label: TTL
Entity: וכב ' השופטת, Label: TTL
Entity: mp, Label: DUC
Entity: קרל גוסטב, Label: DUC
Entity: קרלו, Label: DUC


Processing files:   5%|▌         | 3/55 [00:03<00:52,  1.02s/it]

...


Results for ME-18-05-42697-576.docx:
Entity: עו"ד, Label: TTL
Entity: c, Label: DUC


Processing files:  98%|█████████▊| 54/55 [00:52<00:00,  1.31it/s]


Results for ME-18-07-3742-540.docx:
Entity: עו"ד, Label: TTL
Entity: עו"ד, Label: TTL
Entity: מס' 400, Label: DUC
Entity: השופטת, Label: TTL
Entity: כב' השופט, Label: TTL
Entity: מאזדה 3, Label: DUC


Processing files: 100%|██████████| 55/55 [00:52<00:00,  1.05it/s]


Results for SH-16-11-49772-358.docx:
Entity: עו"ד, Label: TTL
Entity: עו"ד, Label: TTL
Entity: ברטה, Label: DUC
Entity: קארין, Label: DUC
Entity: יריחו, Label: DUC
Entity: קארל גוסטב, Label: DUC
Entity: עו"ד, Label: TTL
Entity: עו"ד, Label: TTL
